### Water Pouring Problem 

In [3]:
# ---------------------------------------
# glass: capacity, current level
# a pair of glass:
# goal: try to reach
# pouring actions: empty, fill, transfer
# solution: sequences of actions
#
# successor: a collection of states that you can reach
# ---------------------------------------
def pour_problem(X, Y, goal, start = (0, 0)):
        """X and Y are thr capacity of glasses; (x, y) is current fill levels
        and represents a state. The goal is a level that can be in either glass.
        Start at start state and following successors until we reach the goal.
        Keep track of frontier and previously explored; fail when no frontier."""
        if goal is start:
            return [start]
        explored = set() # set of states we have visited
        frontier = [ [ start] ] # ordered list of paths we have blazed
        while frontier:
            path = frontier.pop(0)
            (x, y) = path[-1] # last state in the first path of the frontier
            for (state, action) in successors(x, y, X, Y).items():
                if state not in explored:
                    explored.append(state) 
                    path2 = path + [action, state]
                    if goal is state:
                        return path2
                    else:
                        frontier.append(path2)
        return Fail

def successors(x, y, X, Y):
    """Return a dict of {state: action} pairs describing what can be reached from the (x, y)
    state, and how."""
    assert x <= X and y <= Y ##(x, y) is glass level; X and Y are glass sizes
    return {((0, y + x) if y + x <= Y else(x - (Y - y), y + (Y - y))): 'X --> Y',
            ((x + y, 0) if y + x <= X else(x + (X - x), y - (X - x))): 'X <-- Y',
            (X, y): 'fill X', (x, Y): 'fill Y',
            (0, y): 'empty X', (x, 0): 'empty Y'}
    
Fail = []

In [4]:
import doctest  

### Bridge Problem 

In [16]:
#--------------------------------------
# people: 4 
# speed: 1min, 2min, 5min, 10min
# The bridge only can across two people one time
# collection of people: here, there
# tool: torch or the light
# state: (here, there, t)
#--------------------------------------

# -----------------
# User Instructions
# 
# Write a function, bsuccessors(state), that takes a state as input
# and returns a dictionary of {state:action} pairs.
#
# A state is a (here, there, t) tuple, where here and there are 
# frozensets of people (indicated by their times), and potentially
# the 'light,' t is a number indicating the elapsed time.
#
# An action is a tuple (person1, person2, arrow), where arrow is 
# '->' for here to there or '<-' for there to here. When only one 
# person crosses, person2 will be the same as person one, so the
# action (2, 2, '->') means that the person with a travel time of
# 2 crossed from here to there alone.


def bsuccessors(state):
    """Return a dict of {state:action} pairs. A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the 'light', and t is a number indicating the elapsed time. Action is represented
    as a tuple (person1, person2, arrow), where arrow is '->' for here to there and 
    '<-' for there to here."""
    here, there, t = state
    if 'light' in here:
        return dict(((here - frozenset([a, b, 'light']),
                      there | frozenset([a, b, 'light']),
                      t + max(a,b)), (a, b, '->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light')
    else:
        return dict(((here | frozenset([a, b, 'light']),
                      there - frozenset([a, b, 'light']),
                      t + max(a,b)), (a, b, '<-'))
                    for a in there if a is not 'light'
                    for b in there if b is not 'light')


# path_states should return a list of the states. in a path, and 
# path_actions should return a list of the actions.

def path_states(path):
    "Return a list of states in this path."
    return path[0::2]

def path_actions(path):
    "Return a list of actions in this path."
    return path[1::2]

def bridge_problem(here):
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed)
    frontier = [ [(here, frozenset(), 0)] ] # ordered list of paths we have blazed
#     if not here:
#         return frontier[0]

    while frontier:
        path = frontier.pop(0)
        
        ## modify code
        here1, there1, t1 = state1 = path[-1]  ## Check for solution when we pull best path
        if not here1 or here1 == set(['light']):
            return path
        for (state, action) in bsuccessors(path[-1]).items():
            if state not in explored:
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]
#                 if not here: # Thai is, nobody left here
#                     return path2
#                 else:
                frontier.append(path2)
                frontier.sort(key = elapsed_time)
    return []

def elapsed_time(path):
    return path[-1][2]
    
def test():

    testpath = [(frozenset([1, 10]), frozenset(['light', 2, 5]), 5), # state 1
                (5, 2, '->'),                                        # action 1
                (frozenset([10, 5]), frozenset([1, 2, 'light']), 2), # state 2
                (2, 1, '->'),                                        # action 2
                (frozenset([1, 2, 10]), frozenset(['light', 5]), 5),
                (5, 5, '->'), 
                (frozenset([1, 2]), frozenset(['light', 10, 5]), 10),
                (5, 10, '->'), 
                (frozenset([1, 10, 5]), frozenset(['light', 2]), 2),
                (2, 2, '->'), 
                (frozenset([2, 5]), frozenset([1, 10, 'light']), 10),
                (10, 1, '->'), 
                (frozenset([1, 2, 5]), frozenset(['light', 10]), 10),
                (10, 10, '->'), 
                (frozenset([1, 5]), frozenset(['light', 2, 10]), 10),
                (10, 2, '->'), 
                (frozenset([2, 10]), frozenset([1, 5, 'light']), 5),
                (5, 1, '->'), 
                (frozenset([2, 10, 5]), frozenset([1, 'light']), 1),
                (1, 1, '->')]
    
    assert path_states(testpath) == [(frozenset([1, 10]), frozenset(['light', 2, 5]), 5), # state 1
                (frozenset([10, 5]), frozenset([1, 2, 'light']), 2), # state 2
                (frozenset([1, 2, 10]), frozenset(['light', 5]), 5),
                (frozenset([1, 2]), frozenset(['light', 10, 5]), 10),
                (frozenset([1, 10, 5]), frozenset(['light', 2]), 2),
                (frozenset([2, 5]), frozenset([1, 10, 'light']), 10),
                (frozenset([1, 2, 5]), frozenset(['light', 10]), 10),
                (frozenset([1, 5]), frozenset(['light', 2, 10]), 10),
                (frozenset([2, 10]), frozenset([1, 5, 'light']), 5),
                (frozenset([2, 10, 5]), frozenset([1, 'light']), 1)]
    
    assert path_actions(testpath) == [(5, 2, '->'), # action 1
                                      (2, 1, '->'), # action 2
                                      (5, 5, '->'), 
                                      (5, 10, '->'), 
                                      (2, 2, '->'), 
                                      (10, 1, '->'), 
                                      (10, 10, '->'), 
                                      (10, 2, '->'), 
                                      (5, 1, '->'), 
                                      (1, 1, '->')]
    
    assert bsuccessors((frozenset([1, 'light']), frozenset([]), 3)) == {
                (frozenset([]), frozenset([1, 'light']), 4): (1, 1, '->')}

    assert bsuccessors((frozenset([]), frozenset([2, 'light']), 0)) =={
                (frozenset([2, 'light']), frozenset([]), 2): (2, 2, '<-')}
    
    return 'tests pass'

print (test())
print(bridge_problem([1 ,2 ,5 ,10]))

tests pass
[(frozenset({1, 2, 'light', 5, 10}), frozenset(), 0), (2, 1, '->'), (frozenset({10, 5}), frozenset({1, 2, 'light'}), 2), (1, 1, '<-'), (frozenset({1, 10, 'light', 5}), frozenset({2}), 3), (5, 10, '->'), (frozenset({1}), frozenset({2, 10, 5, 'light'}), 13), (2, 2, '<-'), (frozenset({1, 2, 'light'}), frozenset({10, 5}), 15), (2, 1, '->'), (frozenset(), frozenset({1, 2, 'light', 5, 10}), 17)]


In [18]:
class TestBridge: """
>>> elapsed_time(bridge_problem([1,2,5,10]))
17

## There are two equally good solutions
>>> S1 = [(2, 1, '->'), (1, 1, '<-'), (5, 10, '->'), (2, 2, '<-'), (2, 1, '->')]
>>> S2 = [(2, 1, '->'), (2, 2, '<-'), (5, 10, '->'), (1, 1, '<-'), (2, 1, '->')]
>>> path_actions(bridge_problem([1,2,5,10])) in (S1, S2)
True

## Try some other problems
>>> path_actions(bridge_problem([1,2,5,10,15,20]))
[(2, 1, '->'), (1, 1, '<-'), (10, 5, '->'), (2, 2, '<-'), (2, 1, '->'), (1, 1, '<-'), (15, 20, '->'), (2, 2, '<-'), (2, 1, '->')]

>>> path_actions(bridge_problem([1,2,4,8,16,32]))
[(2, 1, '->'), (1, 1, '<-'), (8, 4, '->'), (2, 2, '<-'), (1, 2, '->'), (1, 1, '<-'), (16, 32, '->'), (2, 2, '<-'), (2, 1, '->')]

>>> [elapsed_time(bridge_problem([1,2,4,8,16][:N])) for N in range(6)]
[0, 1, 2, 7, 15, 28]

>>> [elapsed_time(bridge_problem([1,1,2,3,5,8,13,21][:N])) for N in range(8)]
[0, 1, 1, 2, 6, 12, 19, 30]

"""

print (doctest.testmod())

TestResults(failed=0, attempted=0)
